# Additional Resources

Make sure you check out the python docs AND PEP (Python Enhancement Proposals) for whatever you're looking at. For PEP, you will see proposals that are accepted, deferred, rejected and even withdrawn; it's useful to read those that were rejected because there will be good reasons for it. The proposal status is indicated explicitly on the PEP website, for example, if its accepted, it will have a status of 'final'.

Fred Baptiste's recommended books (in no particular order):
- Learning Python (Mark Lutz) : Beginner -> Advanced 
- Fluent Python (Luciano Ramalho) : Intermediate -> Advanced
- Python Cookbook (David Beazley) : Intermediate -> Advanced : How to solve very specific problems
- Effective Python (Brett Slatkin) : Intermediate -> Advanced : How to write idiomatic Python
- Python in a Nutshell (Alex Martelli) : Intermediate -> Advanced : Good as a reference

Fred Baptiste's recommended Youtube content:
- Pycon videos
- Anything by GvR, Raymond Hettinger, Alex Martelli

Websites:
- Planet Python Blog

# 09 - Random Seeds

In random, we can set the seed so that we have reproducibility in whatever we're writing. By default, the seed uses the system time, hence every time you run your program a different seed is set. But we can easily set the seed to something specific. We set the seed using `random.seed(<anything>)`. The seed not only affects the values chosen by a random number generator but also how a shuffle is applied e.g. `random.shuffle`.

# 10 - Random Choices

How do we pick a random element from a list/sequence? Many languages have a random number generator so the common practice is to generate a random number and then access the element associated with that number. In Python we have a builtin approach within the standard library called `random.choice` which takes an iterable.

In [9]:
import random

random.choice(range(1000))

393

If we wanted to generate 5 random numbers then we may use a list comprehension, but really we should check within the standard library first. There's a `random.choices(list, k=)` function. We can even skew the distribution by adding a weights parameter which, for a list of 5 looks like: [1, 1, 1, 1, 1] 

In [10]:
weights = [1] * 100
weights[42] = 250

In [11]:
print(random.choices(range(100), k=20, weights=weights))

[54, 20, 42, 83, 42, 42, 42, 35, 42, 42, 42, 42, 42, 42, 49, 42, 42, 79, 42, 42]


# 11 - Random Samples

We can use `random.sample(list, k=)` to get a sample of `k` values that are all unique.

# 12 - Timing code using timeit

The `timeit` module is similar to our timer decorators that we made except it has a couple of benefits. Firstly, it's platform-specific so its implementation will depend on what's best for your operating system. Also, it can time blocks of code as opposed to just functions. It disables garbage collection and finally it can be run from the commandline. 

In [12]:
from timeit import timeit

help(timeit)

Help on function timeit in module timeit:

timeit(stmt='pass', setup='pass', timer=<built-in function perf_counter>, number=1000000, globals=None)
    Convenience function to create Timer object and call timeit method.



- The `stmt` is the code that we pass for timing. It has to be passed as a string. 
- `number` is how many times to repeat the timing.
- `setup` is a piece of code (str) that gets executed once before the main statement is executed N times. By default its `1_000_000`.
- By default `globals=None` which means that the module has absolutely no idea of any namespace. It starts completely fresh. But, we can pass in a namespace e.g. locals(), globals() or `<module>.__dict__` if we want.
- The return is the **total** time taken for all repeats.

Example: Lets calculate the time it takes to `import math` vs `from math import sqrt`. We need to use the `setup` keyword because in a program, we'll likely only import a module once and then use it many times. It would be unrealistic to import a module before each use of it. So we use `setup` to make it run once.

In [13]:
timeit(stmt= 'math.sqrt(2)', setup='import math') # 1_000_000 repeats.

0.24117028799992113

In [14]:
timeit(stmt= 'sqrt(2)', setup='from math import sqrt') # 1_000_000 repeats. This is only marginally faster

0.19499893500142207

If we want `timeit` to have a starting point for variables in the namespace then we can use the `globals` keyword. Then, we no longer need to import it.

In [15]:
from math import sqrt

print('sqrt' in globals())
timeit(stmt='sqrt(2)', globals = globals())

True


0.21371531299882918

# 13 - Don't Use args and kwargs Names Blindly

If we don't know what will be passed into a function it's okay to use `*args` and `**kwargs`, e.g. in decorator functions. But, if we we're expecting a particular set of keyword arguments e.g. we're creating a `Person` class and we want to allow users to add custom attributes, we would use `**custom_attributes` as the parameter name.

# 14 Python Command Line arguments

The folder in this directory contains 10 examples that are fully annotated. Examples 1-5 using `argv` from `sys`. It's only fine for small simple commands. For more complicated examples, we need to import `argparse`. These are explained in examples 6-10.

# 15 - Sentinel Values for Parameter Defaults

Often we specify the default for a function parameter as `None`. This allows to determine if the user specified an argument for that parameter or not. 

There's a potential issue here!

What happens if we need to differentiate between the following:
* a non-`None` value was provided for the argument
* a `None` value *was* provided for the argument
* the argument was not provided at all

Obviously, if we write our function this way, it will not work as intended:

In [16]:
def validate(a=None):
    if a is not None:
       print('Argument was provided')
    else:
        print('Argument was not provided')
        
validate()
validate(100)
validate(None)

Argument was not provided
Argument was provided
Argument was not provided


Instead, we need to use a different **sentinel** value. But which one?

How can we **guarantee** that whatever sentinel we use will not be explicitly passed in by the user? 

For example we could try to use something like an unlikely string or integer. But that does not guarantee that the caller won't use that precise sentinel value at some point.

The easiest thing to do is to create an instance of the `object` class. This is guaranteed to result in an object that the user cannot pass to us (they have no way of getting their hands on that object - or at least not without the absolute intention to do so). (remember that Python will always allow us to shoot ourselves in the foot if we try hard enough :-) )

In [20]:
_sentinel = object()

def validate(a=_sentinel):
    print(id(a))
    if a is not _sentinel:
        print('Argument was provided')
    else:
        print('Argument was not provided')

validate()
validate(100)
validate(None)

139990344349600
Argument was not provided
139990380629328
Argument was provided
94270042768352
Argument was provided


The reason why this works is because, when we create this `_sentinel` object, it is assigned a reference in memory. Now, this memory id will be forever tied up to this object. So any object that a user creates will have a different memory ID. Therefore, when we check if that new object has the same ID as our object, it will never match. The only way of getting round this is if we delete our sentinel object which frees up the memory address, and by purely chance a user creates a new variable and it gets assigned that memory address. 

Here's the same example but with more arguments:

In [21]:
def validate(a=object(), b=object(), *, kw=object()):
    default_a = validate.__defaults__[0]
    default_b = validate.__defaults__[1]
    default_kw = validate.__kwdefaults__['kw']
    
    if a is not default_a:
        print('Argument a was provided')
    else:
        print('Argument a was not provided')
        
    if b is not default_b:
        print('Argument b was provided')
    else:
        print('Argument b was not provided')
        
    if kw is not default_kw:
        print('Argument kw was provided')
    else:
        print('Argument kw was not provided')
        
validate(100, 200, kw=None)

Argument a was provided
Argument b was provided
Argument kw was provided


# 16 - Simulating a simple Switch in Python

There's an example of a single dispatcher in this subsection. It's using our implementation of single dispatch. It may be worth running the code through pythontutor or something.

In [22]:
def singledispatch(fn):
    registry = dict()
    registry[object] = fn
    
    def register(type_):
        def inner(fn):
            registry[type_] = fn
            return fn  # we do this so we can stack register decorators!
        return inner
   
    def decorator(arg):
        fn = registry.get(type(arg), registry[object])
        return fn(arg)

    decorator.register = register
    return decorator

In [23]:
def switcher(fn):
    registry = dict()
    registry['default'] = fn
    
    def register(case):

        
        def inner(fn):
            registry[case] = fn
            return fn  # we do this so we can stack register decorators!
        return inner
   
    def decorator(case):
        fn = registry.get(case, registry['default'])
        return fn()

    decorator.register = register
    return decorator

In [24]:
@switcher
def dow():
    print('Invalid day of week')
    
@dow.register(1)
def dow_1():
    print('Monday')
    
dow.register(2)(lambda: print('Tuesday'))
dow.register(3)(lambda: print('Wednesday'))
dow.register(4)(lambda: print('Thursday'))
dow.register(5)(lambda: print('Friday'))
dow.register(6)(lambda: print('Saturday'))
dow.register(7)(lambda: print('Sunday'))

<function __main__.<lambda>()>

In [25]:
dow(1)

Monday
